In [ ]:
import numpy as np
import pandas as pd
import yaml
from hipe4ml.model_handler import ModelHandler
import ROOT
ROOT.gSystem.Load('../pdfLib/libRooATan.dylib')
import math
import pickle
import os
import sys
from copy import deepcopy
sys.path.append('../include')
import utils
import para as para
import fRead as fRead
import simultaneous_fit as emFit
import plot as plot
from Data import dataInterval, mcDataInterval, dataGroup, mcDataGroup

#ROOT.Math.IntegratorOneDimOptions.SetDefaultRelTolerance(1.E-16)
ROOT.Math.IntegratorOneDimOptions.SetDefaultIntegrator("GaussLegendre")
#“1-dim integrators”: “Gauss”, “GaussLegendre”, “Adaptive”, “AdaptiveSingular” “NonAdaptive”
utils.set_style()

In [ ]:
########## Set paramters ##########
CENT_BIN_LIST = para.CENT_BIN_LIST
# PT_BIN_LIST = para.PT_BIN_LIST
PT_BIN_LIST = para.PT_BIN_LIST_PERFORMANCE
# PT_BIN_LIST = para.PT_BIN_LIST_NEW
# PT_BIN_LIST = para.PT_BIN_LIST_TEST
MASS_BIN = para.MASS_BIN
MODEL_Eff_LIST = para.MODEL_Eff_LIST

sigpdf_list = ["dscb"]
# bkgpdf_list = ["argus", "Landau", "pol5", "dscb"]
# bkgpdf_list = ["argus", "pol3", "dscb"]
bkgpdf_list = ["dscb"]

is_single_matter_type = False
data_type = "24skimmed"
# data_type = "24skimmed_reduced"
# data_type = "24skimmed_newReduced"
# data_type = "24newSkimmed"

# bkgType = "Sideband"
bkgType = "LikeSign"

pt_dir = utils.convert_ptbin_to_dir(PT_BIN_LIST[0])
# ModelPath = "../MLAna/Model/24skimmed"
# ModelPath = "../MLAna_LikeSign/Model/24skimmed"
# ModelPath = "../MLAna_LikeSign_LooseCut/Model/24skimmed"
# ModelPath = f"../Model/{pt_dir}/24skimmed_{bkgType}"
ModelPath = f"../ModelLooseCut/{pt_dir}/24skimmed_{bkgType}"
# ModelPath = f"../Model/{pt_dir}/backup_LikeSign"

if len(sigpdf_list) == 1 and len(bkgpdf_list) == 1:
    output_prefix = f'Plot/{pt_dir}/{sigpdf_list[0]}_{bkgpdf_list[0]}/'
else:
    output_prefix = f"Plot/{pt_dir}/Merged/"

if not os.path.exists(output_prefix):
    os.makedirs(output_prefix)

score_eff_arrays_dict = pickle.load(
    open(ModelPath + "/file_score_eff_dict", "rb"))

config_file_path = fRead.getConfigPath(data_type)
config_file = open(config_file_path, 'r')
config = yaml.full_load(config_file)

In [ ]:
############### Readin dataset and model ############### 
DataTH = fRead.getDataTH(data_type, period="24amanao")
MCTH = fRead.getMCTH("24skimmed_newReduced", period="25a3")
BkgTH_mixedDeuteron = fRead.getBkgTH("24skimmed_newReduced", "mixed_deuteron", DataTH, period="")
BkgTH_mixedUncorrelated = fRead.getBkgTH("24skimmed_newReduced", "mixed_uncorrelated", DataTH, period="")

DataPDRaw = DataTH.get_data_frame()
MCPDRaw = MCTH.get_data_frame()
BkgPDRaw_mixed_deuteron = BkgTH_mixedDeuteron.get_data_frame()
BkgPDRaw_mixedUncorrelated = BkgTH_mixedUncorrelated.get_data_frame()

utils.calNewElements(DataPDRaw)
utils.calNewElements(BkgPDRaw_mixed_deuteron)
utils.calNewElements(MCPDRaw)
utils.calNewElements(BkgPDRaw_mixedUncorrelated)

DataTH.set_data_frame(DataPDRaw)
BkgTH_mixedDeuteron.set_data_frame(BkgPDRaw_mixed_deuteron)
BkgTH_mixedUncorrelated.set_data_frame(BkgPDRaw_mixed_deuteron)
MCTH.set_data_frame(MCPDRaw)

print("Count of Raw Data: ", len(DataPDRaw))
print("Count of Raw MC: ", len(MCPDRaw))
print("Count of Raw bkg_mixed_deuteron: ", len(BkgPDRaw_mixed_deuteron))
print("Count of Raw bkg_mixed_uncorrelated: ", len(BkgPDRaw_mixedUncorrelated))

if len(CENT_BIN_LIST) != 1:
    raise ValueError("CENT_BIN_LIST should have only one element") # Now we only consider one centrality bin

model_hdl = utils.createEmptyList( [len(CENT_BIN_LIST)] )
for icent, cent_bin in enumerate(CENT_BIN_LIST):
    for pt_bin in PT_BIN_LIST[icent]:
        modelfile_name = ModelPath + '/Model' + "pT" + str(pt_bin[0]) + "_" + str(pt_bin[1])
        modelReadin = ModelHandler()
        modelReadin.load_model_handler(modelfile_name)
        model_hdl[icent].append(deepcopy(modelReadin))

In [ ]:
############### Reweight pt shape ###############
pt_spectrum = fRead.getHypertritonPtShape(data_type)
if PT_BIN_LIST == [[[2,5]]]:
    utils.apply_pt_rejection(MCPDRaw, [pt_spectrum], [[-100, 999]], PT_BIN_LIST, option="READ", path="dfResults/pTReweight_25a3_pT2_5.pkl") # centrality unavailable
elif PT_BIN_LIST == [[[2, 3], [3, 5]]]:
    utils.apply_pt_rejection(MCPDRaw, [pt_spectrum], [[-100, 999]], PT_BIN_LIST, option="READ", path="dfResults/pTReweight_25a3_pT2_3_5.pkl")
MCPDRaw = MCPDRaw.query("rej == False and fSurvivedEventSelection == True")
# FIXME: The raw pt distribution of mixing background is not flat! Reweight not meaningful
# utils.apply_pt_rejection(BkgPDRaw_mixed_deuteron, [pt_spectrum], [[-100, 999]], PT_BIN_LIST, ptcolumn="fPt") # centrality unavailable
# BkgPDRaw_mixed_deuteron = BkgPDRaw_mixed_deuteron.query("rej == False")

print("After pT reweight")
print("MC dataset:",len(MCPDRaw))
print("Generated MC hypertirton within |y| < 0.5:", len(MCPDRaw.query("fIsSignal == 1 and abs(fGenRapidity) < 0.5")))
print("Reconstructed MC hypertirton within 2 <= pT < 5:", len(MCPDRaw.query("fIsSignal == 1 and fPt >= 2 and fPt < 5")))
print("Generated MC hypertirton within |y| < 0.5 and 2 <= pT < 3 GeV/c:", len(MCPDRaw.query("fIsSignal == 1 and abs(fGenRapidity) < 0.5 and fGenPt >= 2 and fGenPt < 3")))
print("Generated MC hypertirton within |y| < 0.5 and 3 <= pT < 5 GeV/c:", len(MCPDRaw.query("fIsSignal == 1 and abs(fGenRapidity) < 0.5 and fGenPt >= 3 and fGenPt < 5")))

In [ ]:
############### Apply precut and add model prediction to data ###############
# print(DataPDRaw.columns)

# minCosPA = 0.995
# maxDCADaughters = 0.1
# preCut =   f"fCt >= 2 and fCt < 40 and \
#             fCosPA > {minCosPA} and \
#             abs(fTPCNSigmaProton) < 3 and abs(fTPCNSigmaBachelor) < 3 and \
#             fDCADaughters < {maxDCADaughters} and \
#             fDCAPionToPV > 0.05"

if "LooseCut" in ModelPath:
    preCut = utils.convert_sel_to_string(config['MLLoosePreSelection'])
else:
    preCut = utils.convert_sel_to_string(config['MLPreSelection'])
print("ML precut:", preCut)

data = dataGroup(DataPDRaw, preCut, CENT_BIN_LIST, PT_BIN_LIST, sigpdf_list, bkgpdf_list, MASS_BIN, data_type, False, model_hdl)
data.addModelPrediction()
mcdata = mcDataGroup(MCPDRaw, preCut, CENT_BIN_LIST, PT_BIN_LIST, sigpdf_list, bkgpdf_list, MASS_BIN, data_type + "MC", False, model_hdl) # cut for pt intervals include fPt > 0
mcdata.addModelPrediction()
bkg_mixed_deuteron = dataGroup(BkgPDRaw_mixed_deuteron, preCut, CENT_BIN_LIST, PT_BIN_LIST, sigpdf_list, bkgpdf_list, MASS_BIN, data_type + "Mixed_deuteron", False, model_hdl)
bkg_mixed_deuteron.addModelPrediction()
bkg_mixed_uncorrelated = dataGroup(BkgPDRaw_mixedUncorrelated, preCut, CENT_BIN_LIST, PT_BIN_LIST, sigpdf_list, bkgpdf_list, MASS_BIN, data_type + "Mixed_uncorrelated", False, model_hdl)
bkg_mixed_uncorrelated.addModelPrediction()

for icent, cent_bin in enumerate(CENT_BIN_LIST):
    for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
        binCutMC = f'(fGenPt >= {pt_bin[0]}) and (fGenPt < {pt_bin[1]})'
        print("gen", len(MCPDRaw.query(f"fIsSignal == 1 and abs(fGenRapidity) < 0.5 and {binCutMC}")))
        print("reco:", len(mcdata.getDF(icent, ipt)))

In [ ]:
############### Fit MC invariant mass distribution to extract the shape of signal ###############
MCFitpara = utils.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0])] ) # only for DSCB
with ROOT.TFile(output_prefix + "MCfit.root", "recreate") as outfile:
    for icent, cent_bin in enumerate(CENT_BIN_LIST):
        for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
            (_signalCount, _signalError, paras) = mcdata.invMFit(icent, ipt, "DSCB", bkgpdf="none", isMC=True, ifDrawStats=False, outfile = outfile)
            MCFitpara[icent][ipt].append(paras)

In [ ]:
############### Fit data ###############
outfileName = "MLsimultaneousfit.root"
with ROOT.TFile(output_prefix + outfileName, "recreate") as outfile:
    (signalCount, signalError, expBkgCount) = data.doSimultaneousFits(bkg_mixed_deuteron, bkg_mixed_uncorrelated, MODEL_Eff_LIST, score_eff_arrays_dict, 
                                                                      mcpara_list=MCFitpara, fit_massbin=[2.96, 3.02], outfile=outfile)

In [ ]:
# Obtain Absorption factor
# AbsorbFactor = fRead.getAbsorpFactor(CENT_BIN_LIST, PT_BIN_LIST, absorp_file = "../CC_file/absorption_histos_3b.root")
AbsorbFactor = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] # To be implemented

############### Calculate efficiency and expected significance ###############
with ROOT.TFile(output_prefix + "MLsimultaneousfit.root", "update") as outfile:
    Efficiency, _ExpCorrectSignal = mcdata.calculateEfficiency(MCPDRaw, pt_spectrum, fRead.getEventNumber(data_type), para.BR_3body)
    mcdata.saveEfficiencyPlots(outfile)
    ExpSignificance = mcdata.getExpSignificance(MODEL_Eff_LIST, expBkgCount)

data.setPreCutEfficiency(Efficiency)

In [ ]:
############### QA for BDT model ###############
with ROOT.TFile(output_prefix + "MLsimultaneousfit.root", "update") as outfile:
    for icent, cent_bin in enumerate(CENT_BIN_LIST):
        outdir_cent = outfile.Get("Cent"+str(cent_bin[0]) + "_" + str(cent_bin[1]))
        for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
            outdir_pt = outdir_cent.Get("pT"+str(pt_bin[0]) + "_" + str(pt_bin[1]))
            binkey = "pT" + str(pt_bin[0]) + "_" + str(pt_bin[1])
            modelOutput_x = score_eff_arrays_dict[binkey][0][0:len(MODEL_Eff_LIST)]
            BDTefficiency_y = score_eff_arrays_dict[binkey][1][0:len(MODEL_Eff_LIST)]
            plot.plot_bdtefficiency_vs_model_output(outdir_pt, modelOutput_x, BDTefficiency_y, pt_bin)

            if len(MODEL_Eff_LIST) > len(BDTefficiency_y):
                BDTefficiency_y = np.append(BDTefficiency_y, 1)

            for isig, sigfunc in enumerate(sigpdf_list):
                for ibkg, bkgfunc in enumerate(bkgpdf_list):
                    binInfo = f"pT{pt_bin[0]}-{pt_bin[1]} GeV/c {sigfunc} {bkgfunc}"
                    significance = np.array(ExpSignificance[icent][ipt][isig][ibkg])
                    significance_times_BDTEff = significance * BDTefficiency_y
                    plot.plot_significance_times_bdtefficiency(outdir_pt, modelOutput_x, significance_times_BDTEff, f"{binkey}_{sigfunc}_{bkgfunc}", binInfo)

data.setBestBDT(ExpSignificance, BDTefficiency_y)


In [ ]:
############### QA and systematical uncertainties ###############
SystUnc = utils.createEmptyList( [len(CENT_BIN_LIST)])
with ROOT.TFile(output_prefix + "MLsimultaneousfit.root", "update") as outfile:
    for icent, cent_bin in enumerate(CENT_BIN_LIST):
        outdir_cent = outfile.Get("Cent"+str(cent_bin[0]) + "_" + str(cent_bin[1]))
        for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
            outdir_pt = outdir_cent.Get("pT"+str(pt_bin[0]) + "_" + str(pt_bin[1]))
            CorrectedSignal = [] #list of corrected signal to calculate systematic uncertainties
            for isig, sigfunc in enumerate(sigpdf_list):
                for ibkg, bkgfunc in enumerate(bkgpdf_list):
                    binkey = f"pT{pt_bin[0]}_{pt_bin[1]}_{sigfunc}_{bkgfunc}" 
                    binInfo = f"pT{pt_bin[0]}-{pt_bin[1]} GeV/c {sigfunc} {bkgfunc}"
                    
                    # Use the BDT threshold with the highest (significance * BDT efficiency) as the central value
                    best_index = data.getBestBDTIndex(icent, ipt, isig, ibkg)
                    SigToBDTefficiency_y = np.array(data.signalCount[icent][ipt][isig][ibkg])/data.bdt_efficiency
                    SigToBDTefficiency_errory = np.array(data.signalError[icent][ipt][isig][ibkg])/data.bdt_efficiency

                    plot.plot_signal_to_bdtefficiency(outdir_pt, MODEL_Eff_LIST, SigToBDTefficiency_y , SigToBDTefficiency_errory, best_index, binkey, binInfo)
                    
                    # Vary BDT efficiency to calculate systematic uncertainties
                    for var in range(-10, 11):
                        if (best_index + var >= len(MODEL_Eff_LIST)) or (best_index + var < 0):
                            continue
                        CorrectedSignal.append( SigToBDTefficiency_y[best_index+var]/(data.getPrecutEfficiency[icent][ipt] * AbsorbFactor[icent][ipt]) )
                        
            # Calculate systematic uncertainties
            binInfo = f"pT{pt_bin[0]}-{pt_bin[1]} GeV/c "
            
            CorrectedYield_y = np.array(CorrectedSignal)/(fRead.getEventNumber(data_type) * para.BR_3body * (pt_bin[1] - pt_bin[0]))
            if not is_single_matter_type:
                CorrectedYield_y = CorrectedYield_y * 2
            h_CorrectedYield = ROOT.TH1F("hCorrectedYield" + binkey, binInfo + ";Corrected Hypertriton yield;Counts", 20, 0.8*np.min(CorrectedYield_y), 1.2*np.max(CorrectedYield_y) )
            for sigVar in CorrectedYield_y:
                h_CorrectedYield.Fill(sigVar)
            outdir_pt.WriteObject(h_CorrectedYield, "hCorrectedYield" + binkey)
            
            # New tree to store the corrected yield with 3 sigma region to central value
            mean = h_CorrectedYield.GetMean()
            rms  = h_CorrectedYield.GetRMS()
            xTTree = np.zeros(1, dtype=np.float64)
            h_CorrectedYieldTree = ROOT.TH1F("hCorrectedYieldTree" + binkey, binInfo + ";Corrected Hypertriton yield;Counts", 20, 0.8*np.min(CorrectedYield_y), 1.2*np.max(CorrectedYield_y) )
            CorrectedYieldTree = ROOT.TTree("CorrectedYield"+binkey, "CorrectedYield Tree")
            CorrectedYieldTree.Branch("CorrectedYield",xTTree,'CorrectedYield/D')
            for sigVar in CorrectedYield_y:
                if abs(sigVar - mean) <= 3 * rms:
                    xTTree[0] = sigVar
                    h_CorrectedYieldTree.Fill(sigVar)
                    CorrectedYieldTree.Fill()
            outdir_pt.WriteObject(h_CorrectedYieldTree, "hCorrectedYieldTree" + binkey)
            outdir_pt.WriteObject(CorrectedYieldTree, "CorrectedYield" + binkey)

            ##### Systematic uncertainties from multi-trials, absorption effects to be added while producing the histogram#####
            SystUnc[icent].append(h_CorrectedYieldTree.GetStdDev())

In [ ]:
# Produce h3body yield histogram
index_sigpdf = len(sigpdf_list) - 1
index_bkgpdf = len(bkgpdf_list) - 1
hStat, hSyst = data.getYieldHist(icent=0, isig=index_sigpdf, ibkg=index_bkgpdf, SystUnc=SystUnc, absorbfactor_list=AbsorbFactor,
                                 f_corr=para.BR_3body * fRead.getEventNumber(data_type), scale_factor=para.BR_3body)

# Read in the h2body yield histogram
h2bodyStat, h2bodySyst = fRead.getH3L2bodyYieldHist(PT_BIN_LIST)

In [ ]:
ROOT.gStyle.SetOptTitle(0)
############### Save final results ###############
with ROOT.TFile(output_prefix + "Results.root", "recreate") as outfile:
    for icent, cent_bin in enumerate(CENT_BIN_LIST):
        for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
            best_index = data.getBestBDTIndex(icent, ipt, index_sigpdf, index_bkgpdf)
            print(best_index)
            print(data.getSignalCount(icent, ipt, index_sigpdf, index_bkgpdf, best_index))
            binkey = f"pT{pt_bin[0]}_{pt_bin[1]}"
            model_threshold = score_eff_arrays_dict[binkey][0][best_index]
            binInfo = f"pT {pt_bin[0]}-{pt_bin[1]} GeV/c BDTEff={round(MODEL_Eff_LIST[best_index],2)}"
            data_se = data.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            bkg_me_deuteron = bkg_mixed_deuteron.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            bkg_me_uncorrelated = bkg_mixed_uncorrelated.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            if ipt == 0:
                (hRawYield, canvas_bkg, canvas_signal, nsignal, err, expNBkg, bkg_peak_value) = emFit.simultaneousFit(data_se, bkg_me_deuteron, bkg_me_uncorrelated, MCFitpara[icent][ipt][0], nBins = 35, ptlims = pt_bin, lowMassLim = 2.96, highMassLim = 3.02, title = binInfo, corr_bkgPdf=bkgfunc, uncorr_bkgPdf="pol1", df_column="fM")
            else:
                (hRawYield, canvas_bkg, canvas_signal, nsignal, err, expNBkg, dummy_var) = emFit.simultaneousFit(data_se, bkg_me_deuteron, bkg_me_uncorrelated, MCFitpara[icent][ipt][0], nBins = 35, ptlims = pt_bin, lowMassLim = 2.96, highMassLim = 3.02, title = binInfo, corr_bkgPdf=bkgfunc, uncorr_bkgPdf="pol1", df_column="fM", corr_bkg_peak=bkg_peak_value)
            canvas_signal.Write()
            canvas_bkg.Write()
            canvas_signal.SaveAs(output_prefix + "Signal_" + binkey + ".png")
            canvas_signal.SaveAs(output_prefix + "Signal_" + binkey + ".eps")
            canvas_bkg.SaveAs(output_prefix + "Bkg_" + binkey + ".png")
            canvas_bkg.SaveAs(output_prefix + "Bkg_" + binkey + ".eps")
            del hRawYield, canvas_bkg, canvas_signal

        ############### Yield by assuming B.R. = 0.4 ###############
        ROOT.gStyle.SetOptStat(0)
        c_HypYields = utils.TCanvas('HypYields','HypYields')
        c_HypYields.cd()
        h_Back_Yields = ROOT.TH2F("h_Back_Yields", ";#it{p}_{T} (GeV/#it{c});B.R. #times #frac{1}{N_{ev}} #frac{dN}{d#it{y}d#it{p}_{T}} (GeV/#it{c})^{-1}", 1, 1.8, 5.2, 1, 0, max(1.*math.pow(10, -9), 1.5*hStat.GetMaximum()) )
        # hStat.GetYaxis().SetTitle( '#frac{1}{N_{ev}} #frac{dN}{d#it{y}d#it{p}_{T}}(GeV/#it{c})^{-1}' )
        hStat.SetMarkerColor(ROOT.kRed)
        hStat.SetLineColor(ROOT.kRed)
        hStat.SetMarkerStyle(8)
        hStat.SetMarkerSize(1.5)
        hSyst.SetFillStyle(0)
        hSyst.SetLineColor(ROOT.kBlack)
        hSyst.SetMarkerColor(ROOT.kBlack)
        hSyst.SetMarkerStyle(20)
        h_Back_Yields.Draw("")
        hStat.Draw("Esame")
        # hSyst.Draw("E2same")

        h2bodyStat.SetLineColor(ROOT.kBlue)
        h2bodyStat.SetMarkerColor(ROOT.kBlue)
        h2bodyStat.SetMarkerStyle(8)
        h2bodyStat.SetMarkerSize(1.5)
        h2bodyStat.Draw("Esame")
        h2bodySyst.SetFillStyle(0)
        h2bodySyst.SetLineColor(ROOT.kBlue)
        h2bodySyst.SetMarkerColor(ROOT.kBlue)
        h2bodySyst.SetMarkerStyle(20)
        # pt_spectrum.Draw("same")
        legend = ROOT.TLegend(0.6, 0.7, 0.9, 0.9)
        # legend.AddEntry(pt_spectrum, "2body mTExpo fit", "l")
        legend.AddEntry(hStat, "3body yield", "p")
        legend.AddEntry(h2bodyStat, "2body yield", "p")
        legend.Draw()
        outfile.WriteObject(c_HypYields, 'HypYields')
        c_HypYields.SaveAs(output_prefix + "HypYields.png")

        ############### Ratio of B.R. ###############
        c_R = utils.TCanvas('c_R','c_R')
        c_R.cd()
        h_Ratio = ROOT.TH2F("h_Ratio", ";#it{p}_{T} (GeV/c);R", 1, 1.8, 5.2, 1, 0.1, 1)
        h_RStat = h2bodyStat.Clone("h_R")
        h_RSyst = h2bodySyst.Clone("h_RSyst")
        # Recorrect the BR of 3-body yield (2-body already corrected while readin)
        h_SumStat = hStat.Clone("h_SumStat")
        h_SumSyst = hSyst.Clone("h_SumSyst")
        # h_SumStat.Scale(0.4)
        # h_SumSyst.Scale(0.4)
        # Calculate the ratio
        h_SumStat.Add(h2bodyStat)
        h_SumSyst.Add(h2bodySyst)
        h_RStat.Divide(h_SumStat)
        h_RSyst.Divide(h_SumSyst)
        # Plot
        h_RStat.SetTitle("")
        h_RStat.GetXaxis().SetTitle( '#it{p}_{T} (GeV/c)' )
        h_RStat.GetYaxis().SetTitle( 'R' )
        h_RStat.SetLineColor(ROOT.kBlue)
        h_RStat.SetMarkerColor(ROOT.kBlue)
        h_RStat.SetMarkerStyle(8)
        h_RStat.SetMarkerSize(1.5)
        h_RSyst.SetFillStyle(1)
        h_RSyst.SetLineColor(ROOT.kBlack)
        h_RSyst.SetMarkerColor(ROOT.kBlack)
        h_RSyst.SetMarkerStyle(20)
        # tf1_R = ROOT.TF1("tf1_R","[0]", 2, 5)
        # h_RStat.Fit("tf1_R")
        h_Ratio.Draw("")
        h_RStat.Draw("Esame")
        # h_RSyst.Draw("E2same")
        outfile.WriteObject(c_R, 'R')
        c_R.SaveAs(output_prefix + "R.png")